In [1]:
!pip install transformers adapters datasets faiss-gpu

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached adapters-1.1.1-py3-none-any.whl.metadata (17 kB)
  Using cached datasets-3.5.1-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached pyarrow-20.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached a

In [2]:
import pandas as pd
import json

Data

In [3]:
from datasets import load_dataset

corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full")
query_data = load_dataset("princeton-nlp/LitSearch", "query", split="full")

In [4]:
def load_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [5]:
spec_0_train = load_from_json('./datasets/spec0_train.json')
spec_0_val = load_from_json('./datasets/spec0_val.json')
spec_0_test = load_from_json('./datasets/spec0_test.json')
spec_1_train = load_from_json('./datasets/spec1_train.json')
spec_1_val = load_from_json('./datasets/spec1_val.json')
spec_1_test = load_from_json('./datasets/spec1_test.json')

In [8]:
train_data = spec_0_train + spec_1_train
val_data = spec_0_val + spec_1_val
test_data = spec_0_test + spec_1_test

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from adapters import AutoAdapterModel
import numpy as np
from tqdm import tqdm
import random
import os

In [9]:
class LitSearchDataset(Dataset):
    def __init__(self, data: list[dict[str, str]], tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['question']

        # 포지티브 샘플 (1개)
        pos_ctx = item['positive_ctxs'][0]
        pos_text = pos_ctx['title'] + self.tokenizer.sep_token + pos_ctx['text']

        # 네거티브 샘플 (3개 중 랜덤 선택)
        neg_ctx = random.choice(item['negative_ctxs'])
        neg_text = neg_ctx['title'] + self.tokenizer.sep_token + neg_ctx['text']

        # 토크나이징
        query_tokens = self.tokenizer(
            question,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        pos_tokens = self.tokenizer(
            pos_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        neg_tokens = self.tokenizer(
            neg_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # 배치 차원 제거
        query_tokens = {k: v.squeeze(0) for k, v in query_tokens.items()}
        pos_tokens = {k: v.squeeze(0) for k, v in pos_tokens.items()}
        neg_tokens = {k: v.squeeze(0) for k, v in neg_tokens.items()}

        return {
            'query': query_tokens,
            'positive': pos_tokens,
            'negative': neg_tokens,
        }

Loss

In [10]:
class TripletMarginLoss(torch.nn.Module):
    def __init__(self, margin=1.0):
        super(TripletMarginLoss, self).__init__()
        self.margin = margin

    def forward(self, query_emb, pos_emb, neg_emb):
        # L2 거리 계산
        pos_dist = torch.norm(query_emb - pos_emb, p=2, dim=1)
        neg_dist = torch.norm(query_emb - neg_emb, p=2, dim=1)

        # max(0, pos_dist - neg_dist + margin) 형태의 손실
        loss = torch.clamp(pos_dist - neg_dist + self.margin, min=0.0)
        return loss.mean()

Model

In [11]:
from enum import Enum
from typing import Dict, Any, List

import torch

class TextType(Enum):
    KEY = 1
    QUERY = 2

class Retrieval:
    def __init__(self, index_name: str, index_type: str) -> None:
        self.index_name = index_name
        self.index_type = index_type

        self.keys = []
        self.values = []

    def __len__(self) -> int:
        return len(self.keys)

    def _get_embeddings(self, textList: list[str], type: TextType, show_progress_bar: bool = False) -> Any:
        raise NotImplementedError

    def _query(self, query_embedding: torch.Tensor, top_k: int = 10) -> List[int]:
        raise NotImplementedError

    def query(self, query_text: str, n: int, return_keys: bool = False) -> List[Any]:
        embedding_query = self._get_embeddings([query_text], TextType.QUERY)
        indices = self._query(embedding_query, n)
        if return_keys:
            results = [(self.keys[i], self.values[i]) for i in indices]
        else:
            results = [self.values[i] for i in indices]
        return results

    def clear(self) -> None:
        self.keys = []
        self.encoded_keys = []
        self.values = []

    def create_index(self, key_value_pairs: Dict[str, int]) -> None:
        if len(self.keys) > 0:
            raise ValueError("Index is not empty. Please create a new index or clear the existing one.")

        for key, value in key_value_pairs.items():
            self.keys.append(key)
            self.values.append(value)


In [18]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel
import torch
from torch.utils.data import DataLoader
import faiss


from tqdm import tqdm
import os
from enum import Enum


class SPECTER2QueryAdapterFinetuner(Retrieval):
    def __init__(self, base_model_name="allenai/specter2_base", device=None):

        self.keys = []
        self.values = []
        self.index = None
        self.faiss_index = None

        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        self.model = AutoAdapterModel.from_pretrained("allenai/specter2_base")

        self.model.load_adapter("allenai/specter2", source="hf", load_as="proximity")
        self.model.load_adapter("allenai/specter2_adhoc_query", source="hf", load_as="adhoc_query")

        for param in self.model.parameters():
            param.requires_grad = False

        for name, param in self.model.named_parameters():
            if "adapters.adhoc_query" in name:
                param.requires_grad = True

        self.model.to(self.device)

    def encode_text(self, input_ids, attention_mask, adapter_type="proximity"):
        """
        adapter_type: query -> "adhoc_query", text -> "proximity"
        """
        self.model.set_active_adapters(adapter_type)

        outputs = self.model(
            input_ids=input_ids.to(self.device),
            attention_mask=attention_mask.to(self.device)
        )
        embeddings = outputs.last_hidden_state[:, 0, :]  # CLS 토큰 임베딩 사용

        return embeddings

    def encode_query(self, query_text):
        self.model.eval()
        with torch.no_grad():
            tokens = self.tokenizer(
                query_text,
                padding='max_length',
                truncation=True,
                max_length=512,
                return_tensors='pt'
            )
            return self.encode_text(
                tokens['input_ids'],
                tokens['attention_mask'],
                adapter_type="adhoc_query"
            )

    def encode_paper(self, title, abstract):
        self.model.eval()
        with torch.no_grad():
            text = title + self.tokenizer.sep_token + abstract
            tokens = self.tokenizer(
                text,
                padding='max_length',
                truncation=True,
                max_length=512,
                return_tensors='pt'
            )
            return self.encode_text(
                tokens['input_ids'],
                tokens['attention_mask'],
                adapter_type="proximity"
            )

    def finetune(self, train_data, val_data=None, output_dir="./specter2_adhoc_query_finetuned",
                 lr=2e-5, batch_size=8, epochs=3, margin=1.0, eval_steps=100,
                 weight_decay=0.01, warmup_ratio=0.1):

        train_dataset = LitSearchDataset(train_data, self.tokenizer)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        optimizer = torch.optim.AdamW(
            [p for p in self.model.parameters() if p.requires_grad],
            lr=lr,
            weight_decay=weight_decay
        )

        total_steps = len(train_loader) * epochs
        warmup_steps = int(total_steps * warmup_ratio)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=lr,
            total_steps=total_steps,
            pct_start=warmup_ratio,
            anneal_strategy='linear'
        )

        triplet_loss = TripletMarginLoss(margin=margin)

        self.model.train()
        global_step = 0
        best_val_loss = float('inf')

        for epoch in range(epochs):
            epoch_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

            for batch in progress_bar:
                # 쿼리, 포지티브, 네거티브 임베딩
                query_emb = self.encode_text(
                    batch['query']['input_ids'],
                    batch['query']['attention_mask'],
                    adapter_type="adhoc_query"
                )

                pos_emb = self.encode_text(
                    batch['positive']['input_ids'],
                    batch['positive']['attention_mask'],
                    adapter_type="proximity"
                )

                neg_emb = self.encode_text(
                    batch['negative']['input_ids'],
                    batch['negative']['attention_mask'],
                    adapter_type="proximity"
                )

                loss = triplet_loss(query_emb, pos_emb, neg_emb)

                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_([p for p in self.model.parameters() if p.requires_grad], 1.0)
                optimizer.step()
                scheduler.step()

                epoch_loss += loss.item()
                progress_bar.set_postfix({"loss": loss.item()})

                global_step += 1
                if val_data is not None and global_step % eval_steps == 0:
                    val_loss = self.evaluate(val_data, batch_size)
                    print(f"Validation Loss: {val_loss:.4f}")

                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        self.save_model(output_dir)
                        print(f"Model saved to {output_dir} (val_loss: {val_loss:.4f})")

                    self.model.train()

            avg_epoch_loss = epoch_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {avg_epoch_loss:.4f}")

        if val_data is None or epochs % eval_steps != 0:
            self.save_model(output_dir)

        return self.model

    def evaluate(self, val_data, batch_size=8):
        val_dataset = LitSearchDataset(val_data, self.tokenizer)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        self.model.eval()
        triplet_loss = TripletMarginLoss(margin=1.0)
        total_loss = 0.0

        with torch.no_grad():
            for batch in val_loader:
                query_emb = self.encode_text(
                    batch['query']['input_ids'],
                    batch['query']['attention_mask'],
                    adapter_type="adhoc_query"
                )

                pos_emb = self.encode_text(
                    batch['positive']['input_ids'],
                    batch['positive']['attention_mask'],
                    adapter_type="proximity"
                )

                neg_emb = self.encode_text(
                    batch['negative']['input_ids'],
                    batch['negative']['attention_mask'],
                    adapter_type="proximity"
                )

                loss = triplet_loss(query_emb, pos_emb, neg_emb)
                total_loss += loss.item()

        return total_loss / len(val_loader)

    def save_model(self, output_dir):
        os.makedirs(output_dir, exist_ok=True)

        self.model.save_adapter(output_dir, "adhoc_query")

        self.tokenizer.save_pretrained(output_dir)

        print(f"어댑터가 {output_dir}에 저장되었습니다.")

    def _get_embeddings(self, textList: list[str], type: TextType, show_progress_bar: bool = True) -> torch.Tensor:
        if type == TextType.KEY:
            self.model.set_active_adapters("proximity")
        else:
            self.model.set_active_adapters("adhoc_query")

        batch_size = 16
        embeddings = []

        should_show_progress = show_progress_bar and (type == TextType.KEY)

        iterator = range(0, len(textList), batch_size)
        if should_show_progress:
            iterator = tqdm(iterator, desc="Processing document embeddings")

        for i in iterator:
            batch_texts = textList[i:i+batch_size]
            encoded = self.tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=512
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**encoded)

            batch_embeddings = outputs.last_hidden_state[:,0,:].cpu()
            embeddings.append(batch_embeddings)

            if i % (batch_size * 10) == 0:
                torch.cuda.empty_cache()

        embeddings = torch.cat(embeddings, dim=0)

        return embeddings

    def clear(self):
        super().clear()
        self.index = None
        self.faiss_index = None

    def create_index(self, key_value_pairs: dict[str, int]) -> None:
        super().create_index(key_value_pairs)
        self.index = self._get_embeddings(self.keys, TextType.KEY)

        # FAISS 인덱스 생성
        vector_dim = self.index.shape[1]
        index_flat = faiss.IndexFlatIP(vector_dim)
        index_vectors = self.index.numpy()
        faiss.normalize_L2(index_vectors)

        # 인덱스에 벡터 추가
        index_flat.add(index_vectors)
        self.faiss_index = index_flat

    def _query(self, query_embedding: torch.Tensor, top_k: int = 10) -> list[int]:
        if self.faiss_index is None:
            raise ValueError("FAISS index has not been created yet. Call create_index first.")

        query_vector = query_embedding.numpy()
        faiss.normalize_L2(query_vector)
        distances, indices = self.faiss_index.search(query_vector, top_k)

        return indices[0].tolist()

    def query(self, query_text: str, n: int, return_keys: bool = False) -> list:
        query_embedding = self._get_embeddings([query_text], TextType.QUERY)
        indices = self._query(query_embedding, n)

        if return_keys:
            results = [(self.keys[i], self.values[i]) for i in indices]
        else:
            results = [self.values[i] for i in indices]

        return results

Train & Test

In [14]:
def calculate_recall(corpus_id_list: list, retrieved_id_list: list, k: int):
    top_k = retrieved_id_list[:k]
    intersection = set(corpus_id_list) & set(top_k)
    return len(intersection) / len(corpus_id_list) if corpus_id_list else 0.0

In [15]:
def evaluate_model(finetuner, test_data, query_data, k_values=[1, 5, 10, 20]):
    # 제 IDE 로컬 환경 pyright이 자꾸 타입 터쳐서 넣은 타입 캐스팅 코드입니다.
    query_data = cast(Dataset, query_data)

    query_df = pd.DataFrame({
        'query': query_data['query'],
        'corpusids': query_data['corpusids']
    })

    test_questions = [item['question'] for item in test_data]
    filtered_query_df = query_df[query_df['query'].isin(test_questions)]

    results = {}
    result = []

    for k in k_values:
        total_recall = 0
        count = 0
        for i, item in enumerate(test_data):
            query = item['question']
            top_k_results = finetuner.query(query, k)

            query_row = filtered_query_df[filtered_query_df['query'] == query]
            if not query_row.empty:
                true_corpus_ids = query_row.iloc[0]['corpusids']
                if isinstance(true_corpus_ids, list):
                    true_corpus_ids_flat = true_corpus_ids
                else:
                    true_corpus_ids_flat = [true_corpus_ids]

                intersection = set(true_corpus_ids_flat) & set(top_k_results)
                recall = len(intersection) / len(true_corpus_ids_flat) if true_corpus_ids_flat else 0

                total_recall += recall
                count += 1

        if count > 0:
            avg_recall = total_recall / count
            results[f'Recall@{k}'] = avg_recall
            result.append(avg_recall)
        else:
            results[f'Recall@{k}'] = 0
            result.append(0)

    return results

In [16]:
from torch.utils.data import Dataset

def get_clean_corpusid(item: dict) -> int:
    return item['corpusid']

def get_clean_title(item: dict) -> str:
    return item['title']

def get_clean_abstract(item: dict) -> str:
    return item['abstract']

def get_clean_title_abstract(item: dict, tokenizer) -> str:
    title = get_clean_title(item)
    abstract = get_clean_abstract(item)
    return title + tokenizer.sep_token + abstract

def create_kv_pairs(data: Dataset, key: str, tokenizer) -> dict:
    return {get_clean_title_abstract(record, tokenizer): get_clean_corpusid(record) for record in data}

In [19]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import cast

import torch
from torch.utils.data import Dataset
from datasets import load_dataset

random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)

print(f"Training: {len(train_data)} samples (Spec 0: {len(spec_0_train)}, Spec 1: {len(spec_1_train)})")
print(f"Validation: {len(val_data)} samples (Spec 0: {len(spec_0_val)}, Spec 1: {len(spec_1_val)})")
print(f"Testing: {len(test_data)} samples (Spec 0: {len(spec_0_test)}, Spec 1: {len(spec_1_test)})")

# 학습 과정
finetuner = SPECTER2QueryAdapterFinetuner()

# 제 IDE 로컬 환경 pyright이 자꾸 타입 터쳐서 넣은 타입 캐스팅 코드입니다.
corpus_clean_data = cast(Dataset, corpus_clean_data)
kv_pairs = create_kv_pairs(corpus_clean_data, "title_abstract", finetuner.tokenizer)
finetuner.clear()
finetuner.create_index(kv_pairs)

finetuner.finetune(
    train_data=train_data,
    val_data=val_data,
    output_dir="./specter2_adhoc_query_finetuned",
    lr=2e-4,
    batch_size=8,
    epochs=5,
    margin=1.0,
    eval_steps=50,
    weight_decay=0.01,
    warmup_ratio=0.1
)

print("Fine-tuning complete!")

Training: 416 samples (Spec 0: 108, Spec 1: 308)
Validation: 61 samples (Spec 0: 16, Spec 1: 45)
Testing: 120 samples (Spec 0: 31, Spec 1: 89)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/5:  96%|█████████▌| 50/52 [00:58<00:05,  2.79s/it, loss=0]      

Validation Loss: 0.0189
어댑터가 ./specter2_adhoc_query_finetuned에 저장되었습니다.
Model saved to ./specter2_adhoc_query_finetuned (val_loss: 0.0189)


...

Epoch 4/5 - Avg Loss: 0.0134


Epoch 5/5:  81%|████████  | 42/52 [00:49<00:27,  2.78s/it, loss=0]     

Validation Loss: 0.0302


Epoch 5/5: 100%|██████████| 52/52 [01:00<00:00,  1.16s/it, loss=0]    

Epoch 5/5 - Avg Loss: 0.0163
어댑터가 ./specter2_adhoc_query_finetuned에 저장되었습니다.
Fine-tuning complete!


In [20]:
print("Evaluating model on test set...")

query_data = load_dataset("princeton-nlp/LitSearch", "query", split="full")
corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full")

# 전체 테스트 세트 평가
overall_performance = evaluate_model(finetuner, test_data, query_data)
print(f"Overall test performance: {overall_performance}")

# Specificity 0 쿼리에 대한 평가
spec_0_performance = evaluate_model(finetuner, spec_0_test, query_data)
print(f"Specificity 0 performance: {spec_0_performance}")

# Specificity 1 쿼리에 대한 평가
spec_1_performance = evaluate_model(finetuner, spec_1_test, query_data)
print(f"Specificity 1 performance: {spec_1_performance}")

Evaluating model on test set...
Overall test performance: {'Recall@1': 0.19166666666666668, 'Recall@5': 0.3905555555555556, 'Recall@10': 0.5113888888888889, 'Recall@20': 0.6255555555555555}
Specificity 0 performance: {'Recall@1': 0.11290322580645161, 'Recall@5': 0.31827956989247314, 'Recall@10': 0.5602150537634408, 'Recall@20': 0.6376344086021505}
Specificity 1 performance: {'Recall@1': 0.19101123595505617, 'Recall@5': 0.3539325842696629, 'Recall@10': 0.5224719101123596, 'Recall@20': 0.6179775280898876}


In [30]:
def evaluate_model_with_analysis(finetuner, test_data, query_data, k_values=[1, 5, 10, 20]):
    """
    Returns:
        tuple: (결과 성능 지표, 실패 케이스 분석 데이터)
    """
    query_data = cast(Dataset, query_data)

    query_df = pd.DataFrame({
        'query': query_data['query'],
        'corpusids': query_data['corpusids']
    })

    test_questions = [item['question'] for item in test_data]
    filtered_query_df = query_df[query_df['query'].isin(test_questions)]

    results = {}
    fail_cases = []
    
    # 가장 큰 k 값에 대해 전체 분석 정보 수집
    max_k = max(k_values)
    
    test_progress = tqdm(enumerate(test_data), total=len(test_data), desc=f"Evaluating & Analyzing")

    for i, item in test_progress:
        query = item['question']
        
        # 원문 쿼리의 specificity 값을 저장
        specificity = item.get('specificity', 'unknown')
        
        # 모델 결과 가져오기 (반환값을 확장하여 전체 결과와, 각 결과별 텍스트 정보 포함)
        top_results = finetuner.query(query, max_k, return_keys=True)
        top_results_ids = [result[1] for result in top_results]  # (key, value) 쌍에서 value만 추출
        top_results_texts = [result[0] for result in top_results]  # (key, value) 쌍에서 key만 추출
        
        query_row = filtered_query_df[filtered_query_df['query'] == query]
        
        # 해당 쿼리를 찾지 못했다면 다음으로 넘어감
        if query_row.empty:
            continue
            
        true_corpus_ids = query_row.iloc[0]['corpusids']
        
        # true_corpus_ids가 리스트가 아니면 리스트로 변환
        if isinstance(true_corpus_ids, list):
            true_corpus_ids_flat = true_corpus_ids
        else:
            true_corpus_ids_flat = [true_corpus_ids]
        
        # 검색 결과에서 실제 corpus ID를 찾지 못한 경우 (Recall@max_k가 0인 경우)
        intersection = set(true_corpus_ids_flat) & set(top_results_ids)
        if not intersection:
            # 실패 케이스 수집
            fail_case = {
                'query': query,
                'specificity': specificity,
                'true_corpus_ids': true_corpus_ids_flat,
                'retrieved_ids': top_results_ids,
                'retrieved_texts': top_results_texts,
                'ranks': {corpus_id: 'not found' for corpus_id in true_corpus_ids_flat}
            }
            fail_cases.append(fail_case)
        else:
            # 일부 정답을 찾았지만 일부는 놓친 경우
            missing_ids = set(true_corpus_ids_flat) - intersection
            if missing_ids:
                # 찾은 corpus ID의 순위와 놓친 ID 정보를 포함한 분석 데이터 생성
                ranks = {}
                for corpus_id in true_corpus_ids_flat:
                    if corpus_id in top_results_ids:
                        ranks[corpus_id] = top_results_ids.index(corpus_id) + 1  # 1-based 순위
                    else:
                        ranks[corpus_id] = 'not found'
                
                fail_case = {
                    'query': query,
                    'specificity': specificity,
                    'true_corpus_ids': true_corpus_ids_flat,
                    'retrieved_ids': top_results_ids,
                    'retrieved_texts': top_results_texts,
                    'ranks': ranks,
                    'partially_found': True
                }
                fail_cases.append(fail_case)
        
        # k 값별로 Recall 계산
        for k in k_values:
            key = f'Recall@{k}'
            if key not in results:
                results[key] = {'total': 0, 'count': 0}
            
            top_k_ids = top_results_ids[:k]
            recall = len(set(true_corpus_ids_flat) & set(top_k_ids)) / len(true_corpus_ids_flat) if true_corpus_ids_flat else 0
            
            results[key]['total'] += recall
            results[key]['count'] += 1

    # 평균 recall 계산
    final_results = {}
    for k in k_values:
        key = f'Recall@{k}'
        if results[key]['count'] > 0:
            final_results[key] = results[key]['total'] / results[key]['count']
        else:
            final_results[key] = 0

    # 실패 케이스를 specificity 별로 분류
    spec_0_fails = [case for case in fail_cases if case['specificity'] == 0]
    spec_1_fails = [case for case in fail_cases if case['specificity'] == 1]
    
    fail_summary = {
        'total_fails': len(fail_cases),
        'specificity_0_fails': len(spec_0_fails),
        'specificity_1_fails': len(spec_1_fails),
        'fail_ratio': len(fail_cases) / len(test_data) if test_data else 0,
        'spec0_fail_ratio': len(spec_0_fails) / len([item for item in test_data if item.get('specificity') == 0]) if test_data else 0,
        'spec1_fail_ratio': len(spec_1_fails) / len([item for item in test_data if item.get('specificity') == 1]) if test_data else 0,
    }
    
    analysis_data = {
        'fail_cases': fail_cases,
        'fail_summary': fail_summary
    }

    return final_results, analysis_data

In [34]:
def analyze_failures(finetuner, query_data):
    # 데이터 불러오기
    test_spec_0 = load_from_json('./datasets/spec0_test.json')
    test_spec_1 = load_from_json('./datasets/spec1_test.json')
    test_data = test_spec_0 + test_spec_1
    
    # 평가 및 정성분석 실행
    results, analysis_data = evaluate_model_with_analysis(finetuner, test_data, query_data)
    
    # 결과 출력
    print(f"Overall performance: {results}")
    print(f"Failure analysis summary: {analysis_data['fail_summary']}")
    
    # 상세 실패 케이스 분석을 위한 정보 출력
    print("\n===== 실패 케이스 샘플 (최대 5개) =====")
    for i, case in enumerate(analysis_data['fail_cases'][:5]):
        print(f"\nCase {i+1}:")
        print(f"Query: {case['query']}")
        print(f"Specificity: {case['specificity']}")
        print(f"True corpus IDs: {case['true_corpus_ids']}")
        print(f"True corpus ID 순위: {case['ranks']}")
        print(f"Retrieved top 3 documents:")
        for j, (doc_id, doc_text) in enumerate(zip(case['retrieved_ids'][:3], case['retrieved_texts'][:3])):
            # 텍스트가 너무 길면 잘라서 표시
            shortened_text = doc_text[:200] + "..." if len(doc_text) > 200 else doc_text
            print(f"  {j+1}. ID: {doc_id}, Text: {shortened_text}")
    
    # 실패 케이스 데이터 저장
    with open("failure_analysis.json", "w", encoding="utf-8") as f:
        json.dump(analysis_data, f, ensure_ascii=False, indent=2)
    
    print("\n상세 분석 결과가 failure_analysis.json에 저장되었습니다.")
    
    return analysis_data

In [32]:
def analyze_patterns(analysis_data):
    fail_cases = analysis_data['fail_cases']
    
    # 1. 쿼리 길이에 따른 분석
    query_lengths = [len(case['query'].split()) for case in fail_cases]
    avg_query_length = sum(query_lengths) / len(query_lengths) if query_lengths else 0
    
    # 2. Specificity별 분석
    spec_0_fails = [case for case in fail_cases if case['specificity'] == 0]
    spec_1_fails = [case for case in fail_cases if case['specificity'] == 1]
    
    # 3. 검색된 문서들과 정답 문서 간의 유사성 분석
    # (이 부분은 문서 내용을 분석해야 하므로 추가 데이터가 필요할 수 있음)
    
    patterns = {
        'query_length_analysis': {
            'min_length': min(query_lengths) if query_lengths else 0,
            'max_length': max(query_lengths) if query_lengths else 0,
            'avg_length': avg_query_length,
            'length_distribution': {
                'short (< 5 words)': len([l for l in query_lengths if l < 5]),
                'medium (5-10 words)': len([l for l in query_lengths if 5 <= l <= 10]),
                'long (> 10 words)': len([l for l in query_lengths if l > 10])
            }
        },
        'specificity_analysis': {
            'spec_0_count': len(spec_0_fails),
            'spec_1_count': len(spec_1_fails),
            'spec_0_avg_query_length': sum(len(case['query'].split()) for case in spec_0_fails) / len(spec_0_fails) if spec_0_fails else 0,
            'spec_1_avg_query_length': sum(len(case['query'].split()) for case in spec_1_fails) / len(spec_1_fails) if spec_1_fails else 0,
        }
    }
    
    print("\n===== 실패 케이스 패턴 분석 =====")
    print(f"평균 쿼리 길이: {patterns['query_length_analysis']['avg_length']:.2f} 단어")
    print(f"쿼리 길이 분포: {patterns['query_length_analysis']['length_distribution']}")
    print(f"Specificity 0 실패 케이스: {patterns['specificity_analysis']['spec_0_count']} (평균 쿼리 길이: {patterns['specificity_analysis']['spec_0_avg_query_length']:.2f})")
    print(f"Specificity 1 실패 케이스: {patterns['specificity_analysis']['spec_1_count']} (평균 쿼리 길이: {patterns['specificity_analysis']['spec_1_avg_query_length']:.2f})")
    
    # 정답 문서와 상위 검색 결과 간의 주제/내용 일치도 분석을 위한 제안
    print("\n실패 케이스의 정답 문서와 검색된 문서 간의 주제/내용 일치도 분석을 위해서는")
    print("문서 전체 내용에 대한 접근과 추가적인 텍스트 분석이 필요합니다.")
    
    return patterns

In [33]:
analysis_data = analyze_failures(finetuner, query_data)

# 패턴 분석 실행
patterns = analyze_patterns(analysis_data)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating & Analyzing:   0%|          | 0/120 [00:00<?, ?it/s]


ValueError: FAISS index has not been created yet. Call create_index first.